## Lecture 7 - KNN, Kernel Method, and Gaussian Process

### 1. K-Nearest Neighbors

KNN (K-Nearest Neighbor) is a simple supervised classification algorithm we can use to assign a class to new data point. It can be used for regression as well, KNN does not make any assumptions on the data distribution, hence it is non-parametric. It keeps all the training data to make future predictions by computing the similarity between an input sample and each training instance.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
iris = datasets.load_iris()
pair = (2, 3)

xlabel = iris.feature_names[pair[0]]
ylabel = iris.feature_names[pair[1]]

iris_x = iris.data[:,pair]
iris_y = iris.target

In [ ]:
def plot_model(model, x, y, label):
    '''
    model: a fitted model
    x, y: two variables, should be numpy arrays
    label: true label
    '''
    x_min = x.min() - 1
    x_max = x.max() + 1
    y_min = y.min() - 1
    y_max = y.max() + 1
    import  matplotlib.pyplot as plt
    from matplotlib import colors
    colDict = {
        'red': [(0, 1, 1), (1, 0.7, 0.7)],
        'green': [(0, 1, 0.5), (1, 0.7, 0.7)],
        'blue': [(0, 1, 0.5), (1, 1, 1)]
    }
    cmap = colors.LinearSegmentedColormap('red_blue_classes', colDict)
    plt.cm.register_cmap(cmap=cmap)
    nx, ny = 200, 200
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),
                         np.linspace(y_min, y_max, ny))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ## plot colormap
    plt.pcolormesh(xx, yy, Z, cmap='red_blue_classes')
    ## plot boundaries
    plt.contour(xx, yy, Z, [0.5], linewidths=1., colors='k')
    plt.contour(xx, yy, Z, [1], linewidths=1., colors='k')
    ## plot scatters and true labels
    plt.scatter(x, y, c=label)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

In [ ]:
knn=KNeighborsClassifier(n_neighbors=6)
knn.get_params()

#### Parameters
- **n_neighbors**: int, default=5
    - Number of neighbors to use by default for kneighbors queries.
- **weights**: {‘uniform’, ‘distance’} or callable, default=’uniform’
    - Weight function used in prediction. Possible values:
        - ‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.
        - ‘distance’ : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.
        - [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.
- **algorithm**: {‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    - Algorithm used to compute the nearest neighbors:
        - ‘ball_tree’ will use BallTree
        - ‘kd_tree’ will use KDTree
        - ‘brute’ will use a brute-force search.
        - ‘auto’ will attempt to decide the most appropriate algorithm based on the values passed to fit method.
    - Note: fitting on sparse input will override the setting of this parameter, using brute force.
- **leaf_size**: int, default=30
    - Leaf size passed to BallTree or KDTree. This can affect the speed of the construction and query, as well as the memory required to store the tree. The optimal value depends on the nature of the problem.
- **p**: int, default=2
    - Power parameter for the Minkowski metric. When p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.
- **metric**: str or callable, default=’minkowski’
    - The distance metric to use for the tree. The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric. For a list of available metrics, see the documentation of DistanceMetric and the metrics listed in sklearn.metrics.pairwise.PAIRWISE_DISTANCE_FUNCTIONS. Note that the “cosine” metric uses cosine_distances. If metric is “precomputed”, X is assumed to be a distance matrix and must be square during fit. X may be a sparse graph, in which case only “nonzero” elements may be considered neighbors.
- **metric_params**: dict, default=None
    - Additional keyword arguments for the metric function.
- **n_jobs**: int, default=None
    - The number of parallel jobs to run for neighbors search. None means 1 unless in a joblib.parallel_backend context. -1 means using all processors. See Glossary for more details. Doesn’t affect fit method.
#### Attributes
- **classes_**: array of shape (n_classes,)
    - Class labels known to the classifier
- **effective_metric_**: str or callble
    - The distance metric used. It will be same as the metric parameter or a synonym of it, e.g. ‘euclidean’ if the metric parameter set to ‘minkowski’ and p parameter set to 2.
- **effective_metric_params_** : dict
    - Additional keyword arguments for the metric function. For most metrics will be same with metric_params parameter, but may also contain the p parameter value if the effective_metric_ attribute is set to ‘minkowski’.
- **n_features_in_** : int
    - Number of features seen during fit.
##### Note (New in version 0.24.)
- **feature_names_in_**: ndarray of shape (n_features_in_,) 
    -  Names of features seen during fit. Defined only when X has feature names that are all strings.
##### Note (New in version 1.0.)
- **n_samples_fit_** : int
    - Number of samples in the fitted data.
- **outputs_2d_** : bool
    - False when y’s shape is (n_samples, ) or (n_samples, 1) during fit otherwise True.

In [ ]:
knn.fit(iris_x,iris_y)
y_pred=knn.predict(iris_x)
metrics.accuracy_score(iris_y,y_pred)

In [ ]:
plt.figure(figsize=(10, 6))
plot_model(knn, iris_x[:, 0], iris_x[:, 1], label=iris_y)
plt.show()

### 2. Kernel Methods

* We can apply any basis functions whose kernel validity is confirmed to form a linearity in a new feature space. 
* From the definition, kernel function $k$ is defined as 
$$k({\bf x},{\bf x'})=\phi({\bf x})^T\phi(\bf x')$$
* In the example, we use a second degree polynomial kernel function on 2-D inputs ${\bf x}$ and ${\bf z}$:
$$ k({\bf x},{\bf z}) = ({\bf x}^T{\bf z})^2 = (x_1z_1 + x_2z_2)^2$$
$$ = x_1^2z_1^2+2x_1x_2z_1z_2 + x_2^2z_2^2 = (x_1^2,\sqrt{2}x_1x_2, x_2^2)(z_1^2,\sqrt{2}z_1z_2,z_2^2)$$
$$ = \phi({\bf x})^T\phi({\bf z})$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = np.array([1,1,2,3,3,6,6,6,9,9,10,11,12,13,16,18])
z = np.array([18,13,9,6,15,11,6,3,5,2,10,5,6,1,3,1])
label = np.array([1,1,1,1,0,0,0,1,0,1,0,0,0,1,0,1])

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(np.column_stack([x,z]),label)
lr.score(np.column_stack([x,z]),label)

In [ ]:
print(lr.coef_)
print(lr.intercept_)

In [ ]:
x_ = range(20)
y_ = -lr.coef_[0,0]/lr.coef_[0,1]*x_ + -lr.intercept_[0]/lr.coef_[0,1]

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.scatter(x, z, c=label, s=60)
plt.plot(x_,y_,linestyle = '--', label = 'Logistic Regression')
plt.xlabel('x')
plt.ylabel('z')
plt.legend()
plt.show()

* Note: The plot above shows that when the logistic regression is applied directly, the hyperplane does not classify the points perfectly.
    * It is expected because ${\bf x}$ and ${\bf y}$ are not linear. 
* In this case, we can apply a kernel function (a second degree polynomial in this case) and transform into a new space. 

In [ ]:
def mapping(X, y):
    x = np.c_[(X, y)]				
    if len(x) >	2:
        x_1 = x[:,0]**2
        x_2 = np.sqrt(2)*x[:,0]*x[:,1]
        x_3 = x[:,1]**2								
    else:
        x_1 = x[0]**2
        x_2 = np.sqrt(2)*x[0]*x[1]
        x_3 = x[1]**2			    
    trans_x = np.array([x_1, x_2, x_3])				
    return trans_x

In [ ]:
x_1  = mapping(x, z)
x_1.shape

In [ ]:
from mpl_toolkits.mplot3d.axes3d import get_test_data
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1,2,1, projection='3d')
ax.scatter(x_1[0], x_1[1], x_1[2], c=label, s=60)
ax.view_init(30, 185)
ax.set_xlabel('$x^2')
ax.set_ylabel('$\sqrt{2}xz$')
ax.set_zlabel('$z^2$')

ax = fig.add_subplot(1,2,2, projection='3d')
ax.scatter(x_1[0], x_1[1], x_1[1], c=label, s=60)
ax.view_init(0, -180)
ax.set_ylim([150,-50])
ax.set_zlim([-10000,10000])
ax.set_xlabel('$x^2')
ax.set_ylabel('$\sqrt{2}xz$')
ax.set_zlabel('$z^2$')
plt.tight_layout()
plt.show()

In [ ]:
lr.fit(x_1.T, label)
lr.score(x_1.T, label)

In [ ]:
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    """
    Isotropic squared exponential kernel.
    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        (m x n) matrix.
    """
    sqdist = np.sum(X1**2) + np.sum(X2**2) + np.sqrt(2) * np.dot(X1, X2.T)
    return sqdist

In [ ]:
kernel(x,z)

In [ ]:
sum(sum(x_1))

## 3. Gaussian Process Implementation

* For detail explanation: see the link below.
http://krasserm.github.io/2018/03/19/gaussian-processes/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, cm
from mpl_toolkits.mplot3d import Axes3D

def plot_gp(mu, cov, X, X_train=None, Y_train=None, samples=[]):
    X = X.ravel()
    mu = mu.ravel()
    uncertainty = 1.96 * np.sqrt(np.diag(cov))
    
    plt.fill_between(X, mu + uncertainty, mu - uncertainty, alpha=0.1)
    plt.plot(X, mu, label='Mean')
    for i, sample in enumerate(samples):
        plt.plot(X, sample, lw=1, ls='--', label=f'Sample {i+1}')
    if X_train is not None:
        plt.plot(X_train, Y_train, 'rx')
    plt.legend()


def plot_gp_2D(gx, gy, mu, X_train, Y_train, title, i):
    ax = plt.gcf().add_subplot(1, 2, i, projection='3d')
    ax.plot_surface(gx, gy, mu.reshape(gx.shape), cmap=cm.coolwarm, linewidth=0, alpha=0.2, antialiased=False)
    ax.scatter(X_train[:,0], X_train[:,1], Y_train, c=Y_train, cmap=cm.coolwarm)
    ax.set_title(title)

### Define Kernel function using squared exponential kernel (also known as Gaussian kernel or RBF kernel)

$$ k(x_i,x_j) = \sigma_f^2\exp\big(-\frac{1}{2l^2}(x_i-x_j)^T(x_i-x_j)\big)$$
* A parameter $l$ controls the smoothness of the function.
* A parameter $\sigma^2$ contrls the vertical variation. 
* The effects of parameters are shown after the posterior prediction.

In [ ]:
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    """
    Isotropic squared exponential kernel.
    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        (m x n) matrix.
    """
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

### 1. Prior generation

* Let the prior over functions be with $\mu=0$ and $\Sigma$ over a Gaussian kernel function parameters with value of $l=1$ and $\sigma^2=1$. 
* The prior will draw random variable samples from the corresponding multivariate normal. 

In [ ]:
# Finite number of points
X = np.arange(-5, 5, 0.2).reshape(-1, 1)

# Mean and covariance of the prior
mu = np.zeros(X.shape)
cov = kernel(X, X)

# Draw three samples from the prior
samples = np.random.multivariate_normal(mu.ravel(), cov, 6)

# Plot GP mean, uncertainty region and samples 
plot_gp(mu, cov, X, samples=samples)

### 2. Posterior computation with noise-free data, $y=f+\epsilon$

* In noise-free case, $\epsilon=0$. $\epsilon\sim{\mathcal N}(\mu=0,\sigma_y I)$ in noisy case.
* We can use Eq. 11 from the lecture notes to implement. 
* Or we can use the noisy case solution with $\sigma_y=0$:
 $$p(f^*|X^*,X,y)={\mathcal N}(f^*|\mu^*,\Sigma^*+\sigma_y^2I)$$
 $$\mu^* = K^{*T}K_y^{-1}y$$
 $$\Sigma^* = K^{**}-K^{*T}K_y^{-1}K^*$$
 * where $K^*=k(X,X^*)$, $K_y = K(X,X) + \sigma_y^2I$, and $K^{**}=k(X^*,X^*)$.

In [ ]:
from numpy.linalg import inv

def posterior(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    """
    Computes the suffifient statistics of the posterior distribution 
    from m training data X_train and Y_train and n new inputs X_s.
    
    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.
    
    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    """
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)
    
    ## Solution
    mu_s = K_s.T.dot(K_inv).dot(Y_train)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    
    return mu_s, cov_s

In [ ]:
# Noise free training data
X_train = np.array([-4, -3, -2, -1, 1]).reshape(-1, 1)
Y_train = np.sin(X_train)

# Compute mean and covariance of the posterior distribution
mu_s, cov_s = posterior(X, X_train, Y_train)

samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, 6)
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train, samples=samples)

### 3. Posterior Computation with noise data. 

In [ ]:
noise = 0.4

# Noisy training data
X_train = np.arange(-3, 4, 1).reshape(-1, 1)
Y_train = np.sin(X_train) + noise * np.random.randn(*X_train.shape)

# Compute mean and covariance of the posterior distribution
mu_s, cov_s = posterior(X, X_train, Y_train, sigma_y=noise)

samples = np.random.multivariate_normal(mu_s.ravel(), cov_s, 6)
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train, samples=samples)

In [ ]:
params = [
    (0.3, 1.0, 0.2),
    (3.0, 1.0, 0.2),
    (1.0, 0.3, 0.2),
    (1.0, 3.0, 0.2),
    (1.0, 1.0, 0.05),
    (1.0, 1.0, 1.5),
]

plt.figure(figsize=(12, 5))

for i, (l, sigma_f, sigma_y) in enumerate(params):
    mu_s, cov_s = posterior(X, X_train, Y_train, l=l, 
                            sigma_f=sigma_f, 
                            sigma_y=sigma_y)
    plt.subplot(3, 2, i + 1)
    plt.subplots_adjust(top=2)
    plt.title(f'l = {l}, sigma_f = {sigma_f}, sigma_y = {sigma_y}')
    plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train)

* Higher $l$ is, smoother the function is (narrower uncertainty regions between data points).
* Higher $\sigma_f$ is, wider ther vertial variation is.
* Higher $\sigma_y$ is, higher chance to avoid overfitting.

### 4. Parameter optimization (Practice)

Parameters, $l, \sigma_f$, and $\sigma_y$ can be optimized using the solutions from slide 39 equation:
$$\ln({\mathcal L}) = -\frac{n}{2}\Big[\ln(2\pi)+\ln(\sigma_f^2)+\ln(|K|)\Big]-\frac{1}{\sigma_f^2}\Big((y-I\mu)^TK^{-1}(y-I\mu)\Big)$$
Complete the following code. 

In [ ]:


from numpy.linalg import cholesky, det
from scipy.linalg import solve_triangular
from scipy.optimize import minimize

def nll_fn(X_train, Y_train, noise, naive=True):
    """
    Returns a function that computes the negative log marginal
    likelihood for training data X_train and Y_train and given
    noise level.

    Args:
        X_train: training locations (m x d).
        Y_train: training targets (m x 1).
        noise: known noise level of Y_train.
        naive: if True use a naive implementation of Eq. (11), if
               False use a numerically more stable implementation.

    Returns:
        Minimization objective.
    """
    
    Y_train = Y_train.ravel()
    
    def nll_naive(theta):
        # Naive implementation of Equation above. Works well for the examples 
        # and is numerically less stable compared to 
        # the implementation in nll_stable below.
        K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        return 0.5 * np.log(det(K)) + \
               0.5 * Y_train.dot(inv(K).dot(Y_train)) + \
               0.5 * len(X_train) * np.log(2*np.pi)
        
    def nll_stable(theta):
        # Numerically more stable implementation of Eq. (11) as described
        # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.
        
        K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        L = cholesky(K)
        
        S1 = solve_triangular(L, Y_train, lower=True)
        S2 = solve_triangular(L.T, S1, lower=False)
        
        return np.sum(np.log(np.diagonal(L))) + \
               0.5 * Y_train.dot(S2) + \
               0.5 * len(X_train) * np.log(2*np.pi)

    if naive:
        return nll_naive
    else:
        return nll_stable

# Minimize the negative log-likelihood w.r.t. parameters l and sigma_f.
# We should actually run the minimization several times with different
# initializations to avoid local minima but this is skipped here for
# simplicity.
res = minimize(nll_fn(X_train, Y_train, noise), [1, 1], 
               bounds=((1e-5, None), (1e-5, None)),
               method='L-BFGS-B')

# Store the optimization results in global variables so that we can
# compare it later with the results from other implementations.
l_opt, sigma_f_opt = res.x

# Compute posterior mean and covariance with optimized kernel parameters and plot the results
mu_s, cov_s = posterior(X, X_train, Y_train, l=l_opt, sigma_f=sigma_f_opt, sigma_y=noise)
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train)

## Scikit-learn Gaussian Process Regression

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=rbf, alpha=noise**2)

# Reuse training data from previous 1D example
gpr.fit(X_train, Y_train)

# Compute posterior mean and covariance
mu_s, cov_s = gpr.predict(X, return_cov=True)

# Obtain optimized kernel parameters
l = gpr.kernel_.k2.get_params()['length_scale']
sigma_f = np.sqrt(gpr.kernel_.k1.get_params()['constant_value'])

# Compare with previous results
assert(np.isclose(l_opt, l))
assert(np.isclose(sigma_f_opt, sigma_f))

# Plot the results
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train)

# Gaussian Process Classification

In [ ]:
from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid

from sklearn.datasets import make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [ ]:
def plot_data_2D(X, t):
    class_1 = np.ravel(t == 1)
    class_0 = np.ravel(t == 0)

    plt.scatter(X[class_1, 0], X[class_1, 1], label='Class 1', marker='x', c='red')
    plt.scatter(X[class_0, 0], X[class_0, 1], label='Class 0', marker='o', edgecolors='blue', facecolors='none')

    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')


def plot_pt_2D(grid_x, grid_y, grid_z):
    plt.contourf(grid_x, grid_y, grid_z, cmap='plasma', alpha=0.3, levels=np.linspace(0, 1, 11))
    plt.colorbar(format='%.2f')


def plot_db_2D(grid_x, grid_y, grid_z, decision_boundary=0.5):
    levels = [decision_boundary]
    cs = plt.contour(grid_x, grid_y, grid_z, levels=levels, colors='black', linestyles='dashed', linewidths=2)
    plt.clabel(cs, fontsize=20)

In [ ]:
np.random.seed(0)

X = np.arange(0, 5, 0.2).reshape(-1, 1)
X_test = np.arange(-2, 7, 0.1).reshape(-1, 1)

a = np.sin(X * np.pi * 0.5) * 2
t = bernoulli.rvs(sigmoid(a))


### 1. Kernel Function.
* Use the same kernel function as done in the regression example. 
* Two additional methods to help the effectiveness of kernel computation. 
    * `K_` computes $K+\nu I$ and the diagonals for the prediction.
    * `W_` computes $\sigma(1-\sigma)$ matrix. 

In [ ]:
def kernel(X1, X2, theta):
    """
    Isotropic squared exponential kernel.

    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).
        theta: Kernel parameters

    Returns:
        (m x n) matrix
    """

    sqdist = np.sum(X1 ** 2, 1).reshape(-1, 1) + np.sum(X2 ** 2, 1) - 2 * np.dot(X1, X2.T)
    
    return theta[1] ** 2 * np.exp(-0.5 / theta[0] ** 2 * sqdist)
    

In [ ]:
def K_(X, theta, diag_only=False, nu=1e-5):
    """Helper to apply kernel function."""
    if diag_only:
        # Specific solution for isotropic 
        # squared exponential kernel.
        return theta[1] ** 2 + nu
    else:
        return kernel(X, X, theta) + nu * np.eye(X.shape[0])

def W_(a):
    """Helper to compute matrix W."""
    r = sigmoid(a) * (1 - sigmoid(a))
    return np.diag(r.ravel())

### 2. Posterior Computation
This will update $a$ in slide 42: $a^{(\tau+1)} = K_a(I-WK_a)^{-1}(t-\sigma+Wa^{\tau})$.
At convergence ${\hat a}=a^{(\tau+1)}$, the Gaussian approximation of $p(a^*|t)$ is given by
$$p(a^*|t)\approx{\mathcal N}(a^*|\mu_a,\sigma_{a^*}^2)$$
with 
$$\mu_{a^*}=k^{*T}(t-\sigma)$$
$$\sigma_{a^*}^2 = k^{**}-k^{*T}(W^{-1}+K_a)^{-1}k^*$$

In [ ]:
def posterior_mode(X, t, K_a, max_iter=10, tol=1e-9):
    """
    Computes the mode of posterior p(a|t).
    """
    a_h = np.zeros_like(t)
    I = np.eye(X.shape[0])

    for i in range(max_iter):
        W = W_(a_h)
        Q_inv = np.linalg.inv(I + W @ K_a)
        a_h_new = (K_a @ Q_inv).dot(t - sigmoid(a_h) + W.dot(a_h))
        a_h_diff = np.abs(a_h_new - a_h)
        a_h = a_h_new

        if not np.any(a_h_diff > tol):
            break

    return a_h

### 3. Kernel parameter optimzation. 
* `nll_fn` estimates the optimal kernel values obtained from the equation of $p(t|\theta)$ in Equation 14 in slide 43. 
* `scipy.optimize.minimize` starts from inital kernel parameter values and `minimize` estimates the gradients of the negative log-likelihood. 

In [ ]:
def nll_fn(X, t):
    """
    Returns the negative log-likelihood function for data X, t.
    """
    
    t = t.ravel()

    def nll(theta):
        K_a = K_(X, theta)
        K_a_inv = np.linalg.inv(K_a)

        # posterior mode depends on theta (via K)
        a_h = posterior_mode(X, t, K_a).ravel()
        W = W_(a_h)

        ll = - 0.5 * a_h.T.dot(K_a_inv).dot(a_h) \
             - 0.5 * np.linalg.slogdet(K_a)[1] \
             - 0.5 * np.linalg.slogdet(W + K_a_inv)[1] \
             + t.dot(a_h) - np.sum(np.log(1.0 + np.exp(a_h)))

        return -ll

    return nll

### 4. Prediction
* `predict_pa` estimates $p(a^*|t)\approx{\mathcal N}(a^*|\mu^*,\sigma^*$).
* `predict_pt` estimates the predictive distribution approximation in slide 43, $p(t^*=1|t)\approx\sigma\Big[\mu_{a^*}\Big(1+\frac{\pi\sigma_{a^*}^2}{8}\Big)\Big]^{-1/2}$, to predict $p(t^*=1|t)=0.5$.


In [ ]:
def predict_a(X_test, X, t, theta):
    """
    Computes the mean and variance of logits at points X_test
    given training data X, t and kernel parameters theta.
    """
    K_a = K_(X, theta)
    K_s = kernel(X, X_test, theta)
    a_h = posterior_mode(X, t, K_a)

    W_inv = np.linalg.inv(W_(a_h))
    R_inv = np.linalg.inv(W_inv + K_a)

    a_test_mu = K_s.T.dot(t - sigmoid(a_h))
    # Compute variances only (= diagonal) instead of full covariance matrix
    a_test_var = K_(X_test, theta, diag_only=True) - np.sum((R_inv @ K_s) * K_s, axis=0).reshape(-1, 1)

    return a_test_mu, a_test_var


def predict_pt(X_test, X, t, theta):
    """
    Computes the probability of t=1 at points X_test
    given training data X, t and kernel parameters theta.
    """
    a_mu, a_var = predict_a(X_test, X, t, theta)
    kappa = 1.0 / np.sqrt(1.0 + np.pi * a_var / 8)
    return sigmoid(kappa * a_mu)

In [ ]:
X, t = make_moons(200, noise=0.5, random_state=1)
t = t.reshape(-1, 1)

plot_data_2D(X, t)
plt.title('2D training dataset')
plt.legend();

In [ ]:
res = minimize(nll_fn(X, t), [1, 1],
               bounds=((1e-3, None), (1e-3, None)),
               method='L-BFGS-B')

theta = res.x

print(f'Optimized theta = [{theta[0]:.3f}, {theta[1]:.3f}], negative log likelihood = {res.fun:.3f}')

In [ ]:
grid_x, grid_y = np.mgrid[-4:4:200j, -4:4:200j]
grid = np.stack([grid_x, grid_y], axis=-1)

In [ ]:
pt_test = predict_pt(grid.reshape(-1, 2), X, t, theta).reshape(*grid_x.shape)

plt.figure(figsize=(9, 7))
plot_pt_2D(grid_x, grid_y, pt_test)
plot_db_2D(grid_x, grid_y, pt_test, decision_boundary=0.5)
plot_data_2D(X, t)
plt.title('Predicted class 1 probabilities')
plt.legend();

## Scikit-learn Gaussian Process Classification

In [ ]:
rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)
gpc = GaussianProcessClassifier(kernel=rbf)

gpc.fit(X, t.ravel())

# Obtain optimized kernel parameters
sklearn_theta_0 = gpc.kernel_.k2.get_params()['length_scale']
sklearn_theta_1 = np.sqrt(gpc.kernel_.k1.get_params()['constant_value'])

print(f'Optimized theta = [{sklearn_theta_0:.3f}, {sklearn_theta_1:.3f}], negative log likelihood = {-gpc.log_marginal_likelihood_value_:.3f}')


In [ ]:
pt_test_gpc = gpc.predict_proba(grid.reshape(-1, 2))[:, 1].reshape(*grid_x.shape)

plt.figure(figsize=(9, 7))
plot_pt_2D(grid_x, grid_y, pt_test_gpc)
plot_db_2D(grid_x, grid_y, pt_test_gpc, decision_boundary=0.5)
plot_data_2D(X, t)
plt.title('Predicted class 1 probabilities (GPC)')
plt.legend();